# VinBigData Chest X-ray Abnormalities Detection

* The aim of this notebook is to demonstrate how to create JPEG images from DICOM images using TensorFlow only. 
* This notebook may be useful to those who wish to use TensorFlow to handle DICOM images and build models using JPEG images (other alternatives include TFRecord).

**Note** 
* It may take a long time to convert all images and the images occupy much space, even if they are resized.
* You may want to use a GPU for this notebook to run successfully (I got errors without it :P). Using PyDICOM instead may be slightly faster.

## Install libraries

In [ ]:
!pip install tensorflow_io

In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tqdm.notebook import tqdm

## Read DICOM images

In [ ]:
# Reading DICOM images

def read_dicom(path):
    image_bytes = tf.io.read_file(path)
    image = tfio.image.decode_dicom_image(
        image_bytes, 
        dtype = tf.uint16
    )
    
    image = tf.squeeze(image, axis = 0)
    
    image = tf.image.resize(
        image, 
        (500, 500), 
        preserve_aspect_ratio = True
    )
    
    image = image - tf.reduce_min(image)
    image = image / tf.reduce_max(image)
    image = tf.cast(image * 255, tf.uint8)
    
    return image

## Save JPEG images

In [ ]:
# Destination folder
destination = "train_jpeg"
os.makedirs("train_jpeg", exist_ok = True)

In [ ]:
# Source folder
source = "../input/vinbigdata-chest-xray-abnormalities-detection/train"

for name in tqdm(sorted(os.listdir(source))[:100]):
    image = read_dicom(os.path.join(source, name))
    image = tf.io.encode_jpeg(
        image, 
        quality = 100, 
        format = 'grayscale'
    )
    
    name = name.replace(".dicom", ".jpeg")
    tf.io.write_file(os.path.join(destination, name), image)